Complete the exercises below For **Assignment #13**.

Load the `ISLR2` and the `tidymodels` packages.

In [5]:
library(ISLR2)
library(tidymodels)



In this assignment we will use the `Default` dataset which includes the default status for credit card customers (`default` variable) in addition to each customer's:

1. credit card balance (`balance` variable),
1. student status (`student` variable), and,
1. income (`income` variable).

In [6]:
Default |> head()

,default,student,balance,income
,<fct>,<fct>,<dbl>,<dbl>
1,No,No,729.5265,44361.625
2,No,Yes,817.1804,12106.135
3,No,No,1073.5492,31767.139
4,No,No,529.2506,35704.494
5,No,No,785.6559,38463.496
6,No,Yes,919.5885,7491.559


We will be modeling `default` with the customer features.

Before we begin let's count how many customers fall into each `default` category.

In [7]:
Default |> count(default)

default,n
<fct>,<int>
No,9667
Yes,333


The data is quite imbalanced. This will be important to keep in mind when we evaluate the performance of our model later. 

Run the code below to create and training data from `Default`. We will use the "test" dataset at the end to get a final evaluation of our best model's accuracy.

In [10]:
Default_split = initial_split(Default, prop = 0.90, strata = default)

Default_train = training(Default_split)
Default_test = testing(Default_split)

Create a logistic regression model called `mod`. Set the engine to `glm` and the mode to `classification`. 

In [11]:
mod <- logistic_reg() |>
  set_engine("glm") |>
  set_mode("classification")



Our data is imbalanced. As such, a naive model that *always* predicts a customer to **not default** would be correct quite often. Let's start by calculating the "accuracy" of a naive model. This will be the baseline accuracy by which we evaluate other models.

In [12]:
# This code calculates the accuracy of a model that always predicts default to be "No"

Default_train |>
    mutate(.pred_naive = factor('No', levels = c('No', 'Yes'))) |>
    accuracy(truth = default, .pred_naive)

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.9678889


Let's use k-fold cross validation to evaluate the performance of a model where the outcome is `default` and the predictors are `income` and `balance`.

To start, use `vfold_cv` to generate 10 validation folds (i.e. set the `v` variable to 10). Set the `strata` argument to `default` so we preserve the distribution of `default` values in each fold.

Creat your folds below and use `glimpse` to look at the output table. Call your output folds tables "folds".

In [13]:
folds <- vfold_cv(Default_train, v = 10, strata = default)
glimpse(folds)



Rows: 10
Columns: 2
$ splits <list> [<vfold_split[8100 x 900 x 9000 x 4]>], [<vfold_split[8100 x 9…
$ id     <chr> "Fold01", "Fold02", "Fold03", "Fold04", "Fold05", "Fold06", "Fo…


The code below fits a model to each of your 10 folds. `collect_metrics` finds the average of evaluation metrics for each of your ten models. 

In [14]:
mod |> 
    fit_resamples(default ~ income + balance, folds) |>
    collect_metrics()

.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.97533333,10,0.001759162,Preprocessor1_Model1
brier_class,binary,0.02050483,10,0.001187071,Preprocessor1_Model1
roc_auc,binary,0.94833489,10,0.006842734,Preprocessor1_Model1


❓How does the model accuracy compare to the naive model from above?

The logistic regression model slightly outperforms the naive model. The naive classifier achieves ~96.8% accuracy simply by always predicting “No,” due to class imbalance.
In contrast, the logistic regression model yields ~97.6% accuracy using income and balance as predictors. Although the improvement in accuracy is modest, the model also has a high ROC-AUC (~0.95), showing that it is meaningfully better at distinguishing customers who default versus those who do not.



Complete the cell below to evaluate a model also includes the `student` variable as as predictor.
1. use `default ~ income + balance + student` as the formula,
2. encode your `student` variable with `step_dummy`, and,
3. don't forget to `prep` your recipe!

In [20]:
rec <- recipe(default ~ income + balance + student, data = Default_train) |> step_dummy(student)
prep(rec)

mod |> fit_resamples(rec, folds) |> collect_metrics()



── Recipe ──────────────────────────────────────────────────────────────────────



── Inputs 

Number of variables by role

outcome:   1
predictor: 3



── Training information 

Training data contained 9000 data points and no incomplete rows.



── Operations 

• Dummy variables from: student | Trained



.metric,.estimator,mean,n,std_err,.config
<chr>,<chr>,<dbl>,<int>,<dbl>,<chr>
accuracy,binary,0.97488889,10,0.001793143,Preprocessor1_Model1
brier_class,binary,0.02044279,10,0.001212031,Preprocessor1_Model1
roc_auc,binary,0.94824179,10,0.006719188,Preprocessor1_Model1


❓Does it appear that the model that includes `student` improves upon the first model with only `income` and `balance` as predictors?

No, the model that includes the student variable does not improve upon the model that only uses income and balance. The cross-validated accuracy remains essentially the same (≈0.975), and the ROC-AUC also stays nearly identical (≈0.948). This indicates that student does not add meaningful predictive information for determining whether a person will default.



Finally, estimate the accuracy of an `default ~ income + balance` model on the test data, `Default_test`. 

❓Does our model outperform a naive model?

In [22]:
final_rec <- recipe(default ~ income + balance, data = Default_train) |> prep()
final_fit <- mod |> fit(default ~ income + balance, data = juice(final_rec))
test_predictions <- predict(final_fit, bake(final_rec, new_data = Default_test))
bind_cols(Default_test, test_predictions) |> accuracy(truth = default, estimate = .pred_class)

.metric,.estimator,.estimate
<chr>,<chr>,<dbl>
accuracy,binary,0.959


Yes. Although the model’s test accuracy of about 95.9% is close to the naive model’s accuracy (≈96.8%), the logistic regression model does outperform the naive model in practice. The naive model achieves high accuracy simply by always predicting “No,” but it completely fails at identifying customers who actually default. In contrast, the logistic regression model makes meaningful predictions about defaulters and shows much stronger discriminatory ability (as shown by its high ROC-AUC). Therefore, even with similar accuracy, the logistic model is much more useful for real decision-making.

